In [10]:
# !pip install python-mnist
#Imports and confusion matrix function
import matplotlib.pyplot as plt
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import scipy.io
import numpy as np
# from mnist import MNIST
from keras.datasets import mnist
import keras

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=True,
                          title=None,
                          cmap=plt.cm.Greens, save_to_file = True):
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    fig, ax = plt.subplots(figsize = (16,16))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
   # if save_to_file:
    #    plt.savefig('Assets/files/' + title + '.pdf')
    return ax

In [12]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()

In [13]:
#Create model and fit
model_mnist = RandomForestClassifier()
model_mnist.fit(train_X.reshape(60000,28*28),train_y)
#Model Score
prediction_mnist = model_mnist.score(test_X.reshape(10000, 28*28), test_y)
print(prediction_mnist)

RandomForestClassifier()

In [9]:
#Load SVHN dataset
train_svhn = scipy.io.loadmat('train_32x32.mat')
test_svhn = scipy.io.loadmat('test_32x32.mat')

FileNotFoundError: [Errno 2] No such file or directory: 'train_32x32.mat'

In [ ]:
#Split train/test 
X_train_svhn = train_svhn['X']
y_train_svhn = train_svhn['y']
X_test_svhn = test_svhn['X']
y_test_svhn = test_svhn['y']

In [ ]:
#Create model and fit

model_svhn = RandomForestClassifier()
X_train_3d = [X_train_svhn[:,:,0,i] for i in range(73257)]
X_train_3d = np.asarray(X_train_3d)
X_train_2d = X_train_3d.transpose(2,0,1).reshape(73257,-1)
model_svhn.fit(X_train_2d,np.ravel(y_train_svhn))

In [ ]:
X_test_3d = [X_test_svhn[:,:,0,i] for i in range(26032 )]
X_test_3d = np.asarray(lista_imgs2)
X_test_2d = X_test_2d.transpose(2,0,1).reshape(26032 ,-1)

In [ ]:
prediction_svhn = model_svhn.score(X_test_2d, np.ravel(y_test_svhn))
print(prediction_svhn)

In [21]:
model = keras.models.Sequential([
    keras.layers.Conv2D(activation='relu', padding="SAME", filters=64, kernel_size=7, input_shape=[28, 28, 1]),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(kernel_size=3, activation='relu', padding="SAME", filters=128),
    keras.layers.Conv2D(kernel_size=3, activation='relu', padding="SAME", filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(kernel_size=3, activation='relu', padding="SAME", filters=256),
    keras.layers.Conv2D(kernel_size=3, activation='relu', padding="SAME", filters=256),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=10, activation='softmax'),
])

In [23]:
# (train_X, train_y), (test_X, test_y)
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(train_X.reshape((-1,28,28,1)), train_y, epochs=10)
score = model.evaluate(test_X, test_y)
X_new = test_X[:10] # pretend we have new images
y_pred = model.predict(X_new)

Epoch 1/10
1875/1875 [==============================] - 145s 77ms/step - loss: 0.5330 - accuracy: 0.8439
Epoch 2/10
1875/1875 [==============================] - 144s 77ms/step - loss: 0.1963 - accuracy: 0.9549
Epoch 3/10
1875/1875 [==============================] - 143s 77ms/step - loss: 0.1436 - accuracy: 0.9667
Epoch 4/10
1875/1875 [==============================] - 140s 75ms/step - loss: 0.1344 - accuracy: 0.9692
Epoch 5/10
1875/1875 [==============================] - 142s 76ms/step - loss: 0.1246 - accuracy: 0.9710
Epoch 6/10
1875/1875 [==============================] - 141s 75ms/step - loss: 0.1127 - accuracy: 0.9759
Epoch 7/10
1875/1875 [==============================] - 140s 75ms/step - loss: 0.1077 - accuracy: 0.9749
Epoch 8/10
1875/1875 [==============================] - 141s 75ms/step - loss: 0.1110 - accuracy: 0.9761
Epoch 9/10
1875/1875 [==============================] - 142s 76ms/step - loss: 0.1073 - accuracy: 0.9769
Epoch 10/10
1875/1875 [==============================] 

ValueError: in user code:

    /usr/local/lib/python3.8/dist-packages/keras/engine/training.py:1330 test_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.8/dist-packages/keras/engine/training.py:1320 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.8/dist-packages/keras/engine/training.py:1313 run_step  **
        outputs = model.test_step(data)
    /usr/local/lib/python3.8/dist-pac

In [ ]:
model.save('CNN_MNIST_v1')